In [ ]:
!pip install langchain langchain-community langchain-qdrant langchain-huggingface langchain-cohere
!pip install pypdf qdrant-client sentence-transformers duckduckgo-search

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools import DuckDuckGoSearchRun
from sentence_transformers import SentenceTransformer

In [ ]:
PDF_DIR = "/content/pdfs"
all_docs = []
for filename in os.listdir(PDF_DIR):
    if filename.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(PDF_DIR, filename))
        docs = loader.load()
        all_docs.extend(docs)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(all_docs)

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name, encode_kwargs={'normalize_embeddings': False})
path_name = "../tmp/langchain_qdrant_multi"
collection_name = "university_kb"
client = QdrantClient(path=path_name)
client.create_collection(collection_name=collection_name, vectors_config=VectorParams(size=768, distance=Distance.COSINE))
vector_store = QdrantVectorStore(client=client, collection_name=collection_name, embedding=embeddings)
vector_store.add_documents(documents=chunks)
retriever = vector_store.as_retriever(search_type='mmr', search_kwargs={'k':3})

In [ ]:
llm = ChatCohere(
    model="command-a-03-2025",
    temperature=0.3,
    max_tokens=300,
    cohere_api_key="YOUR_COHERE_API_KEY"
)

prompt = ChatPromptTemplate.from_template("""You are an assistant that answers students' questions about university details, rules, and guidelines.
Always use the provided context to answer.
If the answer is found in the context, give a clean, concise response.
If the context does not contain the answer, say: 'Not found in university guidelines.'""")

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
web_search = DuckDuckGoSearchRun()

In [ ]:
def answer_with_fallback(query):
    retrieval = retriever.get_relevant_documents(query)
    if len(retrieval) == 0:
        return "🔎 From Internet: " + web_search.run(query)
    else:
        response = retrieval_chain.invoke({"input": query})
        return response["answer"]

In [ ]:
query = "When does the technical fest start?"
print(answer_with_fallback(query))